In [21]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import osmnx as ox
import geopandas as gpd
import osmium
from shapely.geometry import Polygon
import json
import fiona
from shapely.geometry import shape 
import shapely

for module in [pd, gpd]:
    print(module.__name__, module.__version__)

pandas 1.2.4
geopandas 0.9.0


In [11]:
pd.set_option('display.max_columns', None)

In [13]:
%%bash
wget https://download.bbbike.org/osm/extract/planet_-9.192,40.715_-7.611,41.555.osm.pbf \
    --quiet -O map_data/Porto.osm.pbf

In [14]:
%%bash
wget https://download.bbbike.org/osm/extract/planet_-9.89,38.265_-8.309,39.136.osm.pbf \
    --quiet -O map_data/Lisbon.osm.pbf

In [18]:
!ogrinfo map_data/Lisbon.osm.pbf

INFO: Open of `map_data/Lisbon.osm.pbf'
      using driver `OSM' successful.
1: points (Point)
2: lines (Line String)
3: multilinestrings (Multi Line String)
4: multipolygons (Multi Polygon)
5: other_relations (Geometry Collection)


%%bash
ogr2ogr -f "GPKG" \
    map_data/lisbon-amenities.gpkg \
    map_data/Lisbon.osm.pbf \
    -where "amenity is not null" \
    POINTS \
    -nln amenity

gdf_lis_amenities = gpd.read_file("map_data/lisbon-amenities.gpkg", driver='GPKG')
gdf_lis_amenities

In [19]:
%%bash
ogr2ogr -f "GPKG" \
    map_data/lisbon-all.gpkg \
    map_data/Lisbon.osm.pbf \
    POINTS \
    -nln all

0...10...20...30...40...50...60...70...80...90...100 - done.


In [20]:
gdf_lis_pts = gpd.read_file("map_data/lisbon-all.gpkg", drive = 'GPKG')
gdf_lis_pts

,osm_id,name,barrier,highway,ref,address,is_in,place,man_made,other_tags,geometry
0,20629108,Lapa da Serra,None,None,None,None,None,neighbourhood,None,None,POINT (-9.39964 38.95586)
1,20629118,None,None,crossing,None,None,None,None,None,None,POINT (-9.41390 38.96183)
2,21272086,None,None,crossing,None,None,None,None,None,"""crossing""=>""uncontrolled"",""crossing_ref""=>""ze...",POINT (-9.18832 38.74858)
3,21404046,None,None,bus_stop,None,None,None,None,None,None,POINT (-9.45296 38.75312)
4,21404049,None,None,bus_stop,None,None,None,None,None,None,POINT (-9.45655 38.75241)
...,...,...,...,...,...,...,...,...,...,...,...
99115,8641657617,None,None,None,None,None,None,None,windmill,None,POINT (-9.25846 38.83832)
99116,8641657618,None,None,None,None,None,None,None,windmill,None,POINT (-9.26068 38.83765)
99117,8641657619,None,None,None,None,None,None,None,windmill,None,POINT (-9.25440 38.84683)
99118,8641657620,None,None,None,None,None,None,None,windmill,None,POINT (-9.25396 38.84726)


In [21]:
gdf_lis_pts['highway'].unique()

array([None, 'crossing', 'bus_stop', 'motorway_junction',
       'traffic_signals', 'turning_circle', 'services', 'mini_roundabout',
       'speed_camera', 'traffic_signals;crossing', 'give_way', 'elevator',
       'stop', 'turning_loop', 'emergency_access_point', 'street_lamp',
       'trailhead', 'toll_gantry', 'rest_area', 'traffic_mirror',
       'platform', 'steps', 'raceway', 'footway', 'milestone', 'path',
       'crossing;traffic_signals'], dtype=object)

## Other Geometries

### Multipoints - Relations

In [22]:
%%bash
ogr2ogr -f "GPKG" \
    map_data/lisbon_multipoints.gpkg \
    map_data/Lisbon.osm.pbf \
    -nlt MULTIPOINTS \
    -nln multipoints

0...10...20...30...40...50...60...70...80...90...100 - done.


Warning 1: A geometry of type LINESTRING is inserted into layer multipoints of geometry type MULTIPOINT, which is not normally allowed by the GeoPackage specification, but the driver will however do it. To create a conformant GeoPackage, if using ogr2ogr, the -nlt option can be used to override the layer geometry type. This warning will no longer be emitted for this combination of layer and feature geometry type.
Warning 1: A geometry of type MULTILINESTRING is inserted into layer multipoints of geometry type MULTIPOINT, which is not normally allowed by the GeoPackage specification, but the driver will however do it. To create a conformant GeoPackage, if using ogr2ogr, the -nlt option can be used to override the layer geometry type. This warning will no longer be emitted for this combination of layer and feature geometry type.
Warning 1: A geometry of type MULTIPOLYGON is inserted into layer multipoints of geometry type MULTIPOINT, which is not normally allowed by the GeoPackage specif

In [23]:
#Read data
layer_file = "map_data/lisbon_multipoints.gpkg"
collection = list(fiona.open(layer_file,'r'))
df1 = pd.DataFrame(collection)

#Check Geometry
def isvalid(geom):
    try:
        shape(geom)
        return 1
    except:
        return 0
    
df1['isvalid'] = df1['geometry'].apply(lambda x: isvalid(x))
df1 = df1[df1['isvalid'] == 1]
collection = json.loads(df1.to_json(orient='records'))

#Convert to geodataframe
gdf_lis_mp = gpd.GeoDataFrame.from_features(collection)

In [24]:
gdf_lis_mp.geometry.type.unique()

array(['MultiPoint', 'LineString', 'MultiLineString', 'MultiPolygon',
       'GeometryCollection'], dtype=object)

### Polygons - Ways

In [36]:
def isvalid(geom):
    try:
        shape(geom)
        return 1
    except:
        return 0

In [25]:
%%bash
ogr2ogr -f "GPKG" \
    map_data/lisbon_polygons.gpkg \
    map_data/Lisbon.osm.pbf \
    -nlt POLYGONS \
    -nln polygons

0...10...20...30...40...50...60...70...80...90...100 - done.


Warning 1: A geometry of type POINT is inserted into layer polygons of geometry type POLYGON, which is not normally allowed by the GeoPackage specification, but the driver will however do it. To create a conformant GeoPackage, if using ogr2ogr, the -nlt option can be used to override the layer geometry type. This warning will no longer be emitted for this combination of layer and feature geometry type.
Warning 1: A geometry of type LINESTRING is inserted into layer polygons of geometry type POLYGON, which is not normally allowed by the GeoPackage specification, but the driver will however do it. To create a conformant GeoPackage, if using ogr2ogr, the -nlt option can be used to override the layer geometry type. This warning will no longer be emitted for this combination of layer and feature geometry type.
Warning 1: A geometry of type MULTILINESTRING is inserted into layer polygons of geometry type POLYGON, which is not normally allowed by the GeoPackage specification, but the driver w

In [26]:
#Read data
layer_file = "map_data/lisbon_polygons.gpkg"
collection = list(fiona.open(layer_file,'r'))
df1 = pd.DataFrame(collection)

df1['isvalid'] = df1['geometry'].apply(lambda x: isvalid(x))
df1 = df1[df1['isvalid'] == 1]
collection = json.loads(df1.to_json(orient='records'))

#Convert to geodataframe
gdf_lis_poly = gpd.GeoDataFrame.from_features(collection)

### All geometries

In [34]:
%%bash
ogr2ogr -f "GPKG" \
    map_data/lisbon_geometry.gpkg \
    map_data/Lisbon.osm.pbf \
    -nlt POLYGONS \
    -nln polygons

0...10...20...30...40...50...60...70...80...90...100 - done.


Warning 1: A geometry of type POINT is inserted into layer polygons of geometry type POLYGON, which is not normally allowed by the GeoPackage specification, but the driver will however do it. To create a conformant GeoPackage, if using ogr2ogr, the -nlt option can be used to override the layer geometry type. This warning will no longer be emitted for this combination of layer and feature geometry type.
Warning 1: A geometry of type LINESTRING is inserted into layer polygons of geometry type POLYGON, which is not normally allowed by the GeoPackage specification, but the driver will however do it. To create a conformant GeoPackage, if using ogr2ogr, the -nlt option can be used to override the layer geometry type. This warning will no longer be emitted for this combination of layer and feature geometry type.
Warning 1: A geometry of type MULTILINESTRING is inserted into layer polygons of geometry type POLYGON, which is not normally allowed by the GeoPackage specification, but the driver w

In [37]:
#Read data
layer_file = "map_data/lisbon_geometry.gpkg"
collection = list(fiona.open(layer_file,'r'))
df1 = pd.DataFrame(collection)

df1['isvalid'] = df1['geometry'].apply(lambda x: isvalid(x))
df1 = df1[df1['isvalid'] == 1]
collection = json.loads(df1.to_json(orient='records'))

#Convert to geodataframe
gdf_lis_geo = gpd.GeoDataFrame.from_features(collection)

In [41]:
gdf_lis_geo.columns

Index(['geometry', 'osm_id', 'name', 'barrier', 'highway', 'ref', 'address',
       'is_in', 'place', 'man_made', 'other_tags', 'waterway', 'aerialway',
       'z_order', 'type', 'osm_way_id', 'aeroway', 'amenity', 'admin_level',
       'boundary', 'building', 'craft', 'geological', 'historic', 'land_area',
       'landuse', 'leisure', 'military', 'natural', 'office', 'shop', 'sport',
       'tourism'],
      dtype='object')

In [40]:
gdf_lis_geo.geometry.type.value_counts()

Polygon               420865
LineString            195623
Point                  99120
GeometryCollection      3261
MultiLineString          642
dtype: int64

In [50]:
list_of_colums = ['geometry', 
                  'osm_id', 
                  'name', 
                  'amenity', 
                  'barrier', 
                  'building', 
                  'highway', 
                  'landuse', 
                  'man_made', 
                  'natural', 
                  'office']

gdf_lis_poly = gdf_lis_geo[list_of_colums]

In [69]:
def new_desc(geo):
    geo['desc'] = None
    lst_cols = [  'name',
                  'amenity', 
                  'barrier', 
                  'building', 
                  'highway', 
                  'landuse', 
                  'man_made', 
                  'natural', 
                  'office']
    for c, row in gdf_lis_poly.iterrows():
        trues = [row[i] for i in lst_cols if row[i] != None]
        geo.loc[c, 'desc'] = [[trues]]
        print(f'done: {c}')
        
    return geo

In [74]:
gdf_lis_poly.geometry[0].x

-9.3996379

In [ ]:
gdf_lis_poly.to_csv('map_data/gdf_lis_poly.csv', index = False)


In [25]:
df_geo = pd.read_csv('map_data/gdf_lis_geo.csv')
df_geo['geometry'] = df_geo['geometry'].apply(shapely.wkt.loads)
gdf = gpd.GeoDataFrame(df_geo, crs='OGC:CRS84')

In [26]:
gdf

,geometry,osm_id,name,barrier,highway,ref,address,is_in,place,man_made,...,historic,land_area,landuse,leisure,military,natural,office,shop,sport,tourism
0,POINT (-9.39964 38.95586),20629108.0,Lapa da Serra,NaN,NaN,NaN,NaN,NaN,neighbourhood,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,POINT (-9.41390 38.96183),20629118.0,NaN,NaN,crossing,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,POINT (-9.18832 38.74858),21272086.0,NaN,NaN,crossing,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,POINT (-9.45296 38.75312),21404046.0,NaN,NaN,bus_stop,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,POINT (-9.45655 38.75241),21404049.0,NaN,NaN,bus_stop,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
719506,GEOMETRYCOLLECTION (LINESTRING (-9.30623 38.69...,12589376.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
719507,GEOMETRYCOLLECTION (LINESTRING (-9.11842 38.76...,12589451.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
719508,GEOMETRYCOLLECTION (LINESTRING (-9.11842 38.76...,12589452.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
719509,GEOMETRYCOLLECTION (LINESTRING (-9.11888 38.76...,12589453.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
type(gdf.geometry)

geopandas.geoseries.GeoSeries

In [28]:
non_collections = gdf[gdf['geometry'].type != 'GeometryCollection'].reset_index()

In [29]:
non_collections.geometry

0                                 POINT (-9.39964 38.95586)
1                                 POINT (-9.41390 38.96183)
2                                 POINT (-9.18832 38.74858)
3                                 POINT (-9.45296 38.75312)
4                                 POINT (-9.45655 38.75241)
                                ...                        
716143    MULTIPOLYGON (((-9.16965 38.71694, -9.16965 38...
716144    MULTIPOLYGON (((-9.16964 38.71651, -9.16963 38...
716145    MULTIPOLYGON (((-9.16964 38.71666, -9.16964 38...
716146    MULTIPOLYGON (((-9.16965 38.71680, -9.16964 38...
716147    MULTIPOLYGON (((-9.16965 38.71694, -9.16965 38...
Name: geometry, Length: 716148, dtype: geometry

In [30]:
multies = non_collections[non_collections['geometry'].type == 'MultiPolygon']

In [33]:
# import folium
# m = folium.Map([38.74288, -9.16624])

# proj_geo = multies
# folium.Choropleth(geo_data=proj_geo, #geo_lis.geometry.buffer(.0005), 
#                   data=None, 
#                   name="buffered road segments").add_to(m)

# folium.LayerControl().add_to(m)
# m